# Unigram tokenization

*Source: https://huggingface.co/learn/llm-course/en/chapter6/7?fw=pt*

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
#!pip install datasets evaluate transformers[sentencepiece]

In [2]:


corpus = [

    "Hopefully, you will be able to understand how they are trained and generate tokens.",
    "Films adapted from comic books have had plenty of success, whether they're about superheroes (Batman, Superman, Spawn), or geared toward kids (Casper) or the arthouse crowd (Ghost World), but there's never really been a comic book like From Hell before. For starters, it was created by Alan Moore (and Eddie Campbell), who brought the medium to a whole new level in the mid '80s with a 12-part series called The Watchmen. To say Moore and Campbell thoroughly researched the subject of Jack the Ripper would be like saying Michael Jackson is starting to look a little odd. The book (or 'graphic novel', if you will) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes. In other words, don't dismiss this film because of its source. If you can get past the whole comic book thing, you might find another stumbling block in From Hell's directors, Albert and Allen Hughes. Getting the Hughes brothers to direct this seems almost as ludicrous as casting Carrot Top in, well, anything, but riddle me this: who better to direct a film that's set in the ghetto and features really violent street crime than the mad geniuses behind Menace II Society? The ghetto in question is, of course, Whitechapel in 1888 London's East End. It's a filthy, sooty place where the whores (called 'unfortunates') are starting to get a little nervous about this mysterious psychopath who has been carving through their profession with surgical precision. When the first stiff turns up, copper Peter Godley (Robbie Coltrane, The World Is Not Enough) calls in Inspector Frederick Abberline (Johnny Depp, Blow) to crack the case. Abberline, a widower, has prophetic dreams he unsuccessfully tries to quell with copious amounts of absinthe and opium. Upon arriving in Whitechapel, he befriends an unfortunate named Mary Kelly (Heather Graham, Say It Isn't So) and proceeds to investigate the horribly gruesome crimes that even the police surgeon can't stomach. I don't think anyone needs to be briefed on Jack the Ripper, so I won't go into the particulars here, other than to say Moore and Campbell have a unique and interesting theory about both the identity of the killer and the reasons he chooses to slay. In the comic, they don't bother cloaking the identity of the Ripper, but screenwriters Terry Hayes (Vertical Limit) and Rafael Yglesias (Les Misérables) do a good job of keeping him hidden from viewers until the very end. It's funny to watch the locals blindly point the finger of blame at Jews and Indians because, after all, an Englishman could never be capable of committing such ghastly acts. And From Hell's ending had me whistling the Stonecutters song from The Simpsons for days ('who holds back the electric car/who made Steve Guttenberg a star?'). Don't worry - it'll all make sense when you see it. Now onto From Hell's appearance: it's certainly dark and bleak enough, and it's surprising to see how much more it looks like a Tim Burton film than Planet of the Apes did (at times, it seems like Sleepy Hollow 2). The print I saw wasn't completely finished (both color and music had not been finalized, so no comments about Marilyn Manson), but cinematographer Peter Deming (Don't Say a Word) ably captures the dreariness of Victorian-era London and helped make the flashy killing scenes remind me of the crazy flashbacks in Twin Peaks, even though the violence in the film pales in comparison to that in the black-and-white comic. Oscar winner Martin Childs' (Shakespeare in Love) production design turns the original Prague surroundings into one creepy place. Even the acting in From Hell is solid, with the dreamy Depp turning in a typically strong performance and deftly handling a British accent. Ians Holm (Joe Gould's Secret) and Richardson (102 Dalmatians) log in great supporting roles, but the big surprise here is Graham. I cringed the first time she opened her mouth, imagining her attempt at an Irish accent, but it actually wasn't half bad.The film, however, is all good."
]

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

C:\Users\Leonardo\Documents\Projects\natural-language-processing\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

word_freqs

defaultdict(int,
            {'▁Hopefully,': 1,
             '▁you': 5,
             '▁will': 1,
             '▁be': 4,
             '▁able': 1,
             '▁to': 15,
             '▁understand': 1,
             '▁how': 2,
             '▁they': 2,
             '▁are': 2,
             '▁trained': 1,
             '▁and': 18,
             '▁generate': 1,
             '▁tokens.': 1,
             '▁Films': 1,
             '▁adapted': 1,
             '▁from': 3,
             '▁comic': 3,
             '▁books': 1,
             '▁have': 2,
             '▁had': 3,
             '▁plenty': 1,
             '▁of': 14,
             '▁success,': 1,
             '▁whether': 1,
             "▁they're": 1,
             '▁about': 4,
             '▁superheroes': 1,
             '▁(Batman,': 1,
             '▁Superman,': 1,
             '▁Spawn),': 1,
             '▁or': 2,
             '▁geared': 1,
             '▁toward': 1,
             '▁kids': 1,
             '▁(Casper)': 1,
             '▁the': 39,


In [5]:
char_freqs = defaultdict(int)
subwords_freqs = defaultdict(int)
for word, freq in word_freqs.items():
    for i in range(len(word)):
        char_freqs[word[i]] += freq
        # Loop through the subwords of length at least 2
        for j in range(i + 2, len(word) + 1):
            subwords_freqs[word[i:j]] += freq

# Sort subwords by frequency
sorted_subwords = sorted(subwords_freqs.items(), key=lambda x: x[1], reverse=True)
sorted_subwords[:10]

[('▁t', 86),
 ('he', 82),
 ('th', 81),
 ('in', 70),
 ('▁a', 68),
 ('▁th', 61),
 ('er', 55),
 ('the', 53),
 ('an', 46),
 ('▁the', 45)]

In [6]:
token_freqs = list(char_freqs.items()) + sorted_subwords[: 300 - len(char_freqs)]
token_freqs = {token: freq for token, freq in token_freqs}

In [7]:
from math import log

total_sum = sum([freq for token, freq in token_freqs.items()])
model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

In [8]:
def encode_word(word, model):
    best_segmentations = [{"start": 0, "score": 1}] + [
        {"start": None, "score": None} for _ in range(len(word))
    ]
    for start_idx in range(len(word)):
        # This should be properly filled by the previous steps of the loop
        best_score_at_start = best_segmentations[start_idx]["score"]
        for end_idx in range(start_idx + 1, len(word) + 1):
            token = word[start_idx:end_idx]
            if token in model and best_score_at_start is not None:
                score = model[token] + best_score_at_start
                # If we have found a better segmentation ending at end_idx, we update
                if (
                    best_segmentations[end_idx]["score"] is None
                    or best_segmentations[end_idx]["score"] > score
                ):
                    best_segmentations[end_idx] = {"start": start_idx, "score": score}

    segmentation = best_segmentations[-1]
    if segmentation["score"] is None:
        # We did not find a tokenization of the word -> unknown
        return ["<unk>"], None

    score = segmentation["score"]
    start = segmentation["start"]
    end = len(word)
    tokens = []
    while start != 0:
        tokens.insert(0, word[start:end])
        next_start = best_segmentations[start]["start"]
        end = start
        start = next_start
    tokens.insert(0, word[start:end])
    return tokens, score

In [9]:
print(encode_word("Hopefully", model))
print(encode_word("This", model))

(['H', 'o', 'pe', 'f', 'u', 'lly'], 33.449067281658294)
(['Th', 'is'], 13.793032536727)


In [10]:
def compute_loss(model):
    loss = 0
    for word, freq in word_freqs.items():
        _, word_loss = encode_word(word, model)
        loss += freq * word_loss
    return loss

In [11]:
compute_loss(model)

12299.348245663088

In [12]:
import copy


def compute_scores(model):
    scores = {}
    model_loss = compute_loss(model)
    for token, score in model.items():
        # We always keep tokens of length 1
        if len(token) == 1:
            continue
        model_without_token = copy.deepcopy(model)
        _ = model_without_token.pop(token)
        scores[token] = compute_loss(model_without_token) - model_loss
    return scores

In [13]:
scores = compute_scores(model)
#print(scores["ll"])
#print(scores["his"])

In [14]:

percent_to_remove = 0.786
while len(model) > 100:
    scores = compute_scores(model)
    sorted_scores = sorted(scores.items(), key=lambda x: x[1])    # Remove percent_to_remove tokens with the lowest scores.
    for i in range(int(len(model) * percent_to_remove)):
        _ = token_freqs.pop(sorted_scores[i][0])
    total_sum = sum([freq for token, freq in token_freqs.items()])
    model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}
    print(f"New vocabulary size: {len(model)}")
    print("Vocabulary:", model)

New vocabulary size: 65
Vocabulary: {'▁': 1.7842214396135274, 'H': 5.833103627758871, 'o': 2.8249488342063227, 'p': 4.0553304005055555, 'e': 2.3804740724644446, 'f': 4.366766558965444, 'u': 3.8521021588922872, 'l': 3.2491060753266394, 'y': 4.257567267000452, ',': 4.580340659263503, 'w': 4.489368881057776, 'i': 2.947372249419208, 'b': 4.1591271941871994, 'a': 2.879930968623676, 't': 2.697609411829721, 'n': 2.9063642256918314, 'd': 3.617529911754455, 'r': 3.0297432468523358, 's': 3.014705369487795, 'h': 3.1822118404966093, 'g': 4.098502572370764, 'k': 4.852274374747145, '.': 5.139956447198926, 'F': 6.238568735867035, 'm': 3.9359836428729893, 'c': 3.608480076234537, 'v': 5.273487839823448, "'": 4.884023073061725, '(': 5.427638519650706, 'B': 6.93171591642698, 'S': 5.833103627758871, ')': 5.427638519650706, 'C': 6.3721001284915575, 'G': 6.3721001284915575, 'W': 6.3721001284915575, 'A': 6.3721001284915575, 'M': 6.015425184552825, 'E': 6.526250808318816, '8': 6.93171591642698, '0': 6.7085723

In [15]:
def tokenize(text, model):
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in words_with_offsets]
    encoded_words = [encode_word(word, model)[0] for word in pre_tokenized_text]
    return sum(encoded_words, [])


tokens = tokenize("I've downloaded the latest cybersecurity-related infographic that highlights the unhackable blockchain", model)
print(len(tokens))
print(tokens)

103
['▁', 'I', "'", 'v', 'e', '▁', 'd', 'o', 'w', 'n', 'l', 'o', 'a', 'd', 'e', 'd', '▁', 't', 'h', 'e', '▁', 'l', 'a', 't', 'e', 's', 't', '▁', 'c', 'y', 'b', 'e', 'r', 's', 'e', 'c', 'u', 'r', 'i', 't', 'y', '-', 'r', 'e', 'l', 'a', 't', 'e', 'd', '▁', 'i', 'n', 'f', 'o', 'g', 'r', 'a', 'p', 'h', 'i', 'c', '▁', 't', 'h', 'a', 't', '▁', 'h', 'i', 'g', 'h', 'l', 'i', 'g', 'h', 't', 's', '▁', 't', 'h', 'e', '▁', 'u', 'n', 'h', 'a', 'c', 'k', 'a', 'b', 'l', 'e', '▁', 'b', 'l', 'o', 'c', 'k', 'c', 'h', 'a', 'i', 'n']
